<a href="https://colab.research.google.com/github/SPORTNERHSE/SPORTNER/blob/master/_FuzzySearchEn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install fuzzywuzzy

In [0]:
pip install fuzzywuzzy[speedup]

In [0]:
pip install stopwords

In [0]:
pip install langdetect

In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk import punkt
from string import punctuation
from fuzzywuzzy import fuzz
from nltk import word_tokenize
#from stop_words import get_stop_words
from nltk.corpus import stopwords
from langdetect import detect
from string import punctuation 
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Redefining everything from the beginning and separating the df by language of the description**

Definition of a new function that check language

In [0]:
def which_language (text):
  import re
  if bool(re.search('[\u0400-\u04FF]', text)) == True:
    return 'russian'
  else:
    return 'english'

In [0]:
teams_names_in_russian_and_english_df = pd.read_csv("teams names in russian and english.csv")
leagues_names_in_russian_and_english_df = pd.read_csv("leagues names in russian and english.csv")

In [0]:
df = pd.read_csv('result.tar.gz', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df['result.tsv'] = df['result.tsv'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'https\S+', '', regex=True)
df = df.replace('', np.NaN)
df = df.replace('\xa0', np.NaN)
df = df.dropna()
df =  df[df['result.tsv'].str.contains('[A-Za-z]')]
df = df[df['result.tsv'].str.split().str.len().gt(1)]

In [0]:
df = df.dropna()

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.reset_index(drop=True)

In [0]:
df.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
1,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония
2,Украинская команда имеет отличный шанс выйти в...,1.55,Заря забьет 1 или 2 гола: да,1xСтавка,2019-08-15 17:00:00,Лига Европы УЕФА,Заря,ЦСКА София
3,«Сент-Миррен» в родных стенах встретится с «Аб...,2.55,«Абердин» фора (-1),BETCITY,2019-08-11 14:00:00,Премьершип,Сент-Миррен,Абердин
4,На стадионе «Айброкс» состоится встреча между ...,1.97,"«Абердин» фора (+1,5)",Марафон,2019-09-28 14:00:00,Премьершип,Глазго Рейнджерс,Абердин


Loop to create a list of languages of each description

In [0]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
  lang.append(which_language(str(text)))

Based on the language list, we add a new column to the dataframe that contain the language of the description

In [0]:
df['Language'] = lang

Split the dataframe based on the language column

In [0]:
grouped=df.groupby('Language')

Create two new dataframe, each contain just one language, those 
groups do not contain the language column

In [0]:
for name, group in grouped: 
  if name == 'english': 
    english = group.drop('Language',axis=1) 
  else: 
    russian = group.drop('Language',axis=1)

In [0]:
english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_1_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_2_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, leagues_names_in_russian_and_english_df,  how='left', left_on=['league_name_in_russian'], right_on = ['leagues name in Russian'])
english.rename({'unique_leagues_in_english': 'league_name_in_english'}, axis=1, inplace=True)


In [0]:
english.drop(['team name in Russian_x' , 'team name in Russian_y' , 'leagues name in Russian' ] , axis=1 , inplace=True)

In [0]:
# Here we are just re-naming the columns so one check that what's in it and interpret the information easily 
english.rename({'team name in English_x': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)
english.rename({'team name in English_y': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)
english.rename({'leagues names in English': 'league_name_in_english'}, axis=1, inplace=True)

In [0]:
# Here we are just setting the correct data type that english names should be string
english['team_of_season_1_name_in_english']=english['team_of_season_1_name_in_english'].astype(str)
english['team_of_season_2_name_in_english']=english['team_of_season_2_name_in_english'].astype(str)
english['league_name_in_english']=english['league_name_in_english'].astype(str)

In [0]:
english.head(2)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League


**Fuzzy_Searchinging the new datasetes:**

**Fuzzy search by teams 1**

In [0]:
# calculating fuzz ratio to match the two strings we have
#%%time
print("Mistmatch Analysis")
print("**************************************")
tokens = []
teams1_match = np.zeros(english.shape[0])
for i, team1 in enumerate(english['team_of_season_1_name_in_english']):
    if team1 in english['result.tsv'].values[i]:
            txt_tokens = []
            txt_tokens.append(word_tokenize(team1))
            for team1 in txt_tokens:
                for t in team1:
                    if fuzz.ratio(english['team_of_season_1_name_in_english'].values[i], t) > 30:
                        teams1_match[i] = 1
                    else:
                        print(t)
                        print(english['team_of_season_1_name_in_english'].values[i])
                        print("...................................................")

Mistmatch Analysis
**************************************
y
Defensa y Justicia
...................................................
y
Defensa y Justicia
...................................................
y
Defensa y Justicia
...................................................
New
New York Red Bulls
...................................................
Red
New York Red Bulls
...................................................
Red
Urawa Red Diamonds
...................................................
New
New York Red Bulls
...................................................
Red
New York Red Bulls
...................................................
y
Defensa y Justicia
...................................................
Red
Urawa Red Diamonds
...................................................
San
San Jose Earthquakes
...................................................
Da
Vasco Da Gama
...................................................
FC
Los Angeles FC
....................................

In [0]:
english['fuzzy_match_for_teams1'] = teams1_match

In [0]:
all_vals = english['fuzzy_match_for_teams1'].shape[0]

In [0]:
success = english[english['fuzzy_match_for_teams1'] == 1].shape[0]
failure = english[english['fuzzy_match_for_teams1'] == 0].shape[0]
print("Total Values are: ",all_vals)
print("Total success/matched values are: ",success)
print("Total failed/mis-matched values are: ",failure)

Total Values are:  89265
Total success/matched values are:  38571
Total failed/mis-matched values are:  50694


In [0]:
fuzzyline_team1 = success / all_vals
print("Result value is: ",fuzzyline_team1)
print("Result value in percentage is: ",fuzzyline_team1*100 , "%")

Result value is:  0.4320954461435053
Result value in percentage is:  43.209544614350534 %


**Fuzzy search by teams 2**

**Non Ngrams variant:**

In [0]:
print("Mistmatch Analysis")
print("**************************************")
#%%time
tokens = []
teams2_match = np.zeros(english.shape[0])
for i, team2 in enumerate(english['team_of_season_2_name_in_english']):
  if team2 in english['result.tsv'].values[i]:
    txt_tokens = []
    txt_tokens.append(word_tokenize(team2))
    for team2 in txt_tokens:
        for t in team2:
            if fuzz.ratio(english['team_of_season_2_name_in_english'].values[i], t) > 35:
                teams2_match[i] = 1
            else:
                print(t)
                print(english['team_of_season_1_name_in_english'].values[i])
                print("...................................................")

Mistmatch Analysis
**************************************
RB
Benfica
...................................................
San
Los Angeles FC
...................................................
Jose
Los Angeles FC
...................................................
and
Greece
...................................................
Tel
CFR Cluj
...................................................
RB
Bayer
...................................................
U20
Ukraine U20
...................................................
U20
Ukraine U20
...................................................
and
Panama
...................................................
Los
Seattle Saunders
...................................................
New
New York Red Bulls
...................................................
Red
Jubilo Iwata
...................................................
New
New York city
...................................................
St
Celtic
...................................................

In [0]:
english['fuzzy_match1_for_teams2'] = teams2_match

In [0]:
all_vals = english['fuzzy_match1_for_teams2'].shape[0]

In [0]:
success = english[english['fuzzy_match1_for_teams2'] == 1].shape[0]
failure = english[english['fuzzy_match1_for_teams2'] == 0].shape[0]
print("Total Values are: ",all_vals)
print("Total success/matched values are: ",success)
print("Total failed/mis-matched values are: ",failure)

Total Values are:  89265
Total success/matched values are:  37116
Total failed/mis-matched values are:  52149


In [0]:
fuzzyline_team2 = success / all_vals

In [0]:
print("Result value is: ",fuzzyline_team2)
print("Result value in percentage is: ",fuzzyline_team2*100 , "%")

Result value is:  0.4157956645941859
Result value in percentage is:  41.57956645941859 %


**Ngrams Variant:**

In [0]:
#%%time 
print("Mistmatch Analysis")
print("**************************************")
teams2_match = np.zeros(english.shape[0])
for i, team2 in tqdm(enumerate(english['team_of_season_2_name_in_english'])):
    n_words_team2 = len(word_tokenize(english['result.tsv'].values[i]))
    #print(n_words_league)
    N = n_words_team2
    words = word_tokenize(team2)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(english['team_of_season_2_name_in_english'].values[i], joined_gram) > 45:
            #print(english[team_of_season_2_name_in_russian'].values[i], joined_gram)
            teams2_match[i] = 1
        else:
                print(joined_gram)
                print(english['team_of_season_2_name_in_english'].values[i])
                print("...................................................")

0it [00:00, ?it/s]

Mistmatch Analysis
**************************************


89265it [00:39, 2274.55it/s]


In [0]:
english['fuzzy_match2_for_teams2'] = teams2_match

In [0]:
all_vals = english['fuzzy_match2_for_teams2'].shape[0]
success = english[english['fuzzy_match2_for_teams2'] == 1].shape[0]
failure = english[english['fuzzy_match2_for_teams2'] == 0].shape[0]

In [0]:
fuzzyline_team2 = success / all_vals
print("Result value is: ",fuzzyline_team2)
print("Result value in percentage is: ",fuzzyline_team2*100 , "%")

Result value is:  0.0016467820534363973
Result value in percentage is:  0.16467820534363972 %


**Fuzzy search by league**

**Non Ngrams Variant:**

In [0]:
#%%time
print("Mistmatch Analysis")
print("**************************************")
tokens = []
leagues_match = np.zeros(english.shape[0])
for i, league in enumerate(english['league_name_in_english']):
  if league in english['result.tsv'].values[i]:
    txt_tokens = []
    txt_tokens.append(word_tokenize(league))
    for league in txt_tokens:
        for t in league:
            if fuzz.ratio(english['league_name_in_english'].values[i], t) > 30:
                leagues_match[i] = 1
            else:
                print(t)
                print(english['league_name_in_english'].values[i])
                print("...................................................")

Mistmatch Analysis
**************************************
Cup
African Cup of Nations
...................................................
of
African Cup of Nations
...................................................
Cup
African Cup of Nations
...................................................
of
African Cup of Nations
...................................................
Cup
African Cup of Nations
...................................................
of
African Cup of Nations
...................................................
A
Serie A
...................................................
A
Serie A
...................................................
A
Serie A
...................................................
A
Serie A
...................................................
A
Serie A
...................................................
Cup
African Cup of Nations
...................................................
of
African Cup of Nations
...................................................
A
Se

In [0]:
english['fuzzy_match1_for_leagues'] = leagues_match

In [0]:
all_vals = english['fuzzy_match1_for_leagues'].shape[0]
success = english[english['fuzzy_match1_for_leagues'] == 1].shape[0]
failure = english[english['fuzzy_match1_for_leagues'] == 0].shape[0]

print("Total Values are: ",all_vals)
print("Total success/matched values are: ",success)
print("Total failed/mis-matched values are: ",failure)


Total Values are:  89265
Total success/matched values are:  3544
Total failed/mis-matched values are:  85721


In [0]:
fuzzyline_leagues = success / all_vals
print("Result value is: ",fuzzyline_leagues)
print("Result value in percentage is: ",fuzzyline_leagues*100 , "%")

Result value is:  0.03970201086652103
Result value in percentage is:  3.970201086652103 %


**Ngrams Variant**

In [0]:
%%time 
leagues_match = np.zeros(english.shape[0])
for i, leagues in tqdm(enumerate(english['league_name_in_english'])):
    n_words_league = len(word_tokenize(english['result.tsv'].values[i]))
    #print(n_words_league)
    N = n_words_league
    words = word_tokenize(leagues)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(english['league_name_in_english'].values[i], joined_gram) > 80:
            #print(english['league_name_in_russian'][i].values, joined_gram)
            leagues_match[i] = 1

89265it [00:40, 2215.43it/s]

CPU times: user 40 s, sys: 261 ms, total: 40.3 s
Wall time: 40.3 s


In [0]:
english['fuzzy_match2_for_leagues'] = leagues_match

In [0]:
all_vals = english['fuzzy_match2_for_leagues'].shape[0]
print(all_vals)
success = english[english['fuzzy_match2_for_leagues'] == 1].shape[0]
print(success)
fuzzyline_leagues = success / all_vals
print(fuzzyline_leagues)

89265
593
0.0066431412087604325


In [0]:
fuzzyline_leagues = success / all_vals
print("Result value is: ",fuzzyline_leagues)
print("Result value in percentage is: ",fuzzyline_leagues*100 , "%")

Result value is:  0.0066431412087604325
Result value in percentage is:  0.6643141208760432 %


**Fuzzy search on bookmakers**

**Non Ngramm Variant**

In [0]:
%%time
tokens = []
bookmakers_match = np.zeros(english.shape[0])
for i, bookmaker in enumerate(english['bookmaker_name']):
  if bookmaker in english['result.tsv'].values[i]:
    txt_tokens = []
    txt_tokens.append(word_tokenize(bookmaker))
    for bookmaker in txt_tokens:
        for t in bookmaker:
            if fuzz.ratio(english['bookmaker_name'].values[i], t) > 90:
                bookmakers_match[i] = 1

CPU times: user 329 ms, sys: 975 µs, total: 329 ms
Wall time: 340 ms


In [0]:
english['fuzzy_match1_for_bookmakers'] = bookmakers_match

In [0]:
all_vals = english['fuzzy_match1_for_bookmakers'].shape[0]
print(all_vals)
success = english[english['fuzzy_match1_for_bookmakers'] == 1].shape[0]
print(success)
fuzzyline_bookmaker = success / all_vals
print(fuzzyline_bookmaker)

89265
4
4.4810396011874754e-05


In [0]:
fuzzyline_bookmaker = success / all_vals
print("Result value is: ",fuzzyline_bookmaker)
print("Result value in percentage is: ",fuzzyline_bookmaker*100 , "%")

Result value is:  4.4810396011874754e-05
Result value in percentage is:  0.004481039601187476 %



**Ngram Variant**


In [0]:
%%time 
bookmakers_match = np.zeros(english.shape[0])
for i, bookmaker in tqdm(enumerate(english['bookmaker_name'])):
    n_words_bookmaker = len(word_tokenize(english['result.tsv'].values[i]))
    #print(n_words_bookmaker)
    N = n_words_bookmaker
    words = word_tokenize(bookmaker)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(english['bookmaker_name'].values[i], joined_gram) > 80:
            #print(english['bookmaker_name'].values[i], joined_gram)
            bookmakers_match[i] = 1

89265it [00:38, 2301.26it/s]

CPU times: user 38.6 s, sys: 248 ms, total: 38.8 s
Wall time: 38.8 s


In [0]:
english['fuzzy_match2_for_bookmakers'] = bookmakers_match

In [0]:
all_vals = english['fuzzy_match2_for_bookmakers'].shape[0]
print(all_vals)
success = english[english['fuzzy_match2_for_bookmakers'] == 1].shape[0]
print(success)
fuzzyline_bookmaker = success / all_vals
print(fuzzyline_bookmaker)

89265
0
0.0


In [0]:
fuzzyline_bookmaker = success / all_vals
print("Result value is: ",fuzzyline_bookmaker)
print("Result value in percentage is: ",fuzzyline_bookmaker*100 , "%")

Result value is:  0.0
Result value in percentage is:  0.0 %



**Fuzzy search by tip_name (betting text):**

**Ngrams Variant** 


In [0]:
%%time 
tipname_match = np.zeros(english.shape[0])
for i, tipname in tqdm(enumerate(english['tip_name'])):
    n_words_tipname = len(word_tokenize(english['result.tsv'].values[i]))
    #print(n_words_tipname)
    N = n_words_tipname
    words = word_tokenize(tipname)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(english['tip_name'].values[i], joined_gram) > 80:
            #print(english['tip_name'].values[i], joined_gram)
            tipname_match[i] = 1

89265it [00:39, 2249.15it/s]

CPU times: user 39.4 s, sys: 261 ms, total: 39.7 s
Wall time: 39.7 s


In [0]:
english['fuzzy_match_for_tipname'] = tipname_match

In [0]:
all_vals = english['fuzzy_match_for_tipname'].shape[0]
print(all_vals)
success = english[english['fuzzy_match_for_tipname'] == 1].shape[0]
print(success)
fuzzyline_tipname = success / all_vals
print(fuzzyline_tipname)

89265
264
0.0029574861367837337


In [0]:
fuzzyline_tipname = success / all_vals
print("Result value is: ",fuzzyline_tipname)
print("Result value in percentage is: ",fuzzyline_tipname*100 , "%")

Result value is:  0.0029574861367837337
Result value in percentage is:  0.29574861367837335 %



**Fuzzy search by match_date_time**

**Non Ngrams Variant:**


In [0]:
%%time
tokens = []
date_match = np.zeros(english.shape[0])
for i, date in enumerate(english['match_date_time']):
  if date in english['result.tsv'].values[i]:
    txt_tokens = []
    txt_tokens.append(word_tokenize(date))
    for date in txt_tokens:
        for t in date:
            if fuzz.ratio(english['match_date_time'].values[i], t) > 30:
                date_match[i] = 1
                print(english['match_date_time'].values[i], t)

CPU times: user 313 ms, sys: 1.98 ms, total: 315 ms
Wall time: 319 ms


In [0]:
english['fuzzy_match_for_date'] = date_match

In [0]:
all_vals = english['fuzzy_match_for_date'].shape[0]
print(all_vals)
success = english[english['fuzzy_match_for_date'] == 1].shape[0]
print(success)
fuzzyline_date = success / all_vals
print(fuzzyline_date)

89265
0
0.0


In [0]:
fuzzyline_date = success / all_vals
print("Result value is: ",fuzzyline_date)
print("Result value in percentage is: ",fuzzyline_date*100 , "%")

Result value is:  0.0
Result value in percentage is:  0.0 %


**Ngrams Variant:**

In [0]:
%%time 
date_match = np.zeros(english.shape[0])
for i, date in tqdm(enumerate(english['match_date_time'])):
    n_words_date = len(word_tokenize(english['result.tsv'].values[i]))
    #print(n_words_date)
    N = n_words_date
    words = word_tokenize(date)
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(i)
    #print(grams)
    for gram in grams: 
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if fuzz.ratio(english['match_date_time'].values[i], joined_gram) > 50:
            #print(english['match_date_time'].values[i], joined_gram)
            date_match[i] = 1

89265it [00:41, 2173.75it/s]

CPU times: user 40.7 s, sys: 290 ms, total: 41 s
Wall time: 41.1 s


In [0]:
english['fuzzy_match_for_date'] = date_match

In [0]:
all_vals = english['fuzzy_match_for_date'].shape[0]
print(all_vals)
success = english[english['fuzzy_match_for_date'] == 1].shape[0]
print(success)
fuzzyline_tipname = success / all_vals
print(fuzzyline_date)

89265
608
0.0


In [0]:
fuzzyline_date = success / all_vals
print("Result value is: ",fuzzyline_date)
print("Result value in percentage is: ",fuzzyline_date*100 , "%")

Result value is:  0.0068111801938049625
Result value in percentage is:  0.6811180193804962 %



**Fuzzy search by coefficients**

**Non Ngrams Variant:**.


In [0]:
%%time
tokens = []
coef_match = np.zeros(english.shape[0])
for i, coef in enumerate(english['odd']):
   if str(coef) in english['result.tsv'].values[i]:
    t=coef
    #txt_tokens = []
    #txt_tokens.append(word_tokenize(coef))
    #for coef in txt_tokens:
    #    for t in coef:
    if fuzz.ratio(english['odd'].values[i], t) > 60:
                coef_match[i] = int(1)
                #print(english['odd'].values[i], t)


CPU times: user 352 ms, sys: 0 ns, total: 352 ms
Wall time: 358 ms


In [0]:


english['fuzzy_match_for_odd'] = coef_match



In [0]:
all_vals = english['fuzzy_match_for_odd'].shape[0]
print(all_vals)
success = english[english['fuzzy_match_for_odd'] == 1].shape[0]
print(success)
fuzzyline_odd = success / all_vals
print(fuzzyline_odd)

89265
1890
0.02117291211561082


In [0]:
fuzzyline_odd = success / all_vals
print("Result value is: ",fuzzyline_odd)
print("Result value in percentage is: ",fuzzyline_odd*100 , "%")

Result value is:  0.02117291211561082
Result value in percentage is:  2.1172912115610822 %
